# Data Science - Laboratorio 5
## Minería de Texto: Clasificación de tweets 
- - -
### Integrantes:
- Diego Alberto Leiva 21752
- José Pablo Orellana 21970
- - -

## Librerias

NOTA: Importante tener el Token de la API en el folder `C:\Users\<Windows-username>\.kaggle\kaggle.json`

Para una guia mas detallada o si tiene otro sistema dirigase a: https://www.kaggle.com/docs/api#getting-started-installation-&-authentication 

Asimismo puede visitar la documentacion oficial de la API: https://github.com/Kaggle/kaggle-api/tree/main/docs#api-credentials

In [1]:
# Kaggle API
import kaggle
from kaggle.rest import ApiException

# Operaciones con archivos
import os
import zipfile

# Operaciones con datos
import pandas as pd

## Carga de Dataset

### Descargar el archivo con Kaggle

Puede omitir esta celda si decide descargar los archivos manualmente desde https://www.kaggle.com/c/nlp-getting-started/data

In [2]:
# Autenticación con Kaggle API
kaggle.api.authenticate()

# Definir competencia y directorio de descarga
competition_name = 'nlp-getting-started'
target_path = 'data'

# Verificar si existen archivos descargados previamente
if  not os.path.exists(f"{target_path}/train.csv") and not os.path.exists(f"{target_path}/test.csv"):
    # Descargar archivos de la competencia
    try:
        kaggle.api.competition_download_files(competition='nlp-getting-started',
                                            path='data',
                                            quiet=False,
                                            force=False)
        
        print(f"EXITO: Archivos descargados en '{target_path}'")

    except ApiException as e:
        print(f"ERROR al llamar KaggleApi -> competition_download \n\t{e}")
        print("Puede que necesite aceptar las reglas de la competencia en https://www.kaggle.com/c/nlp-getting-started/rules")
else:
    print("AVISO: Se encontraron archivos descargados previamente")
    print(f"\tSi desea descargar nuevamente los archivos, elimine '{target_path}/{competition_name}.zip'")

AVISO: Se encontraron archivos descargados previamente
	Si desea descargar nuevamente los archivos, elimine 'data/nlp-getting-started.zip'


### Descomprimir el Zip

Puede omitir esta celda si ya tiene los archivos descomprimidos

In [3]:
# Verificar si ya se descomprimieron los archivos .zip previamente
if  os.path.exists(f"{target_path}/train.csv") and os.path.exists(f"{target_path}/test.csv"):
    print("AVISO: Se encontraron archivos descargados previamente")
    print("\tOmitiendo descompresión de archivos .zip")

# Descomprimir archivos .zip
else:
    try:
        # Verificar si se descargó un archivo .zip y descomprimirlo
        if os.path.exists(f"{target_path}/{competition_name}.zip"):
            # Descomprimir archivo .zip
            with zipfile.ZipFile(f'{target_path}/{competition_name}.zip', 'r') as zip_ref:
                zip_ref.extractall(target_path)

            # Eliminar archivo .zip
            os.remove(f'{target_path}/{competition_name}.zip')
            os.remove(f'{target_path}/sample_submission.csv')
            print(f"EXITO: Archivo .zip descomprimido en '{target_path}'")
        else:
            raise Exception(f"No se encontró archivo .zip en '{target_path}'")

    except Exception as e:
        print(f"ERROR al descomprimir archivo .zip \n\t{e}")

AVISO: Se encontraron archivos descargados previamente
	Omitiendo descompresión de archivos .zip


### Generar dataframes

In [4]:
# Cargar archivos en DataFrames
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

## Comprensión Inicial

### Visualizar Dataframe

In [5]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


### Información General del Dataset

In [6]:
def get_DF_info(df):
    """
    Función que genera un dataframe con información sobre las columnas de un dataframe dado.

    Args:
        df (pandas.DataFrame): El dataframe de entrada.
        
    Returns:
        None
    """
    # Crear un dataframe con la información de las columnas    
    info = pd.DataFrame({
        'Nombre Columna': df.columns,
        'Tipo de Dato': df.dtypes,
        'Valores Nulos (%)': (df.isnull().sum() * 100 / df.shape[0]).round(2),
        'Valores Únicos': df.nunique(),
        'Valores Duplicados': df.duplicated().sum()
    }).reset_index(drop=True)

    # Mostrar la informacion
    print(f"\nEl dataframe tiene {df.shape[0]} filas (registros) y {df.shape[1]} columnas (variables)\n")
    display(info)

In [7]:
print(f"Informacion del dataset: train.csv")
get_DF_info(df=train_df)

Informacion del dataset: train.csv

El dataframe tiene 7613 filas (registros) y 5 columnas (variables)



,Nombre Columna,Tipo de Dato,Valores Nulos (%),Valores Únicos,Valores Duplicados
0,id,int64,0.00,7613,0
1,keyword,object,0.80,221,0
2,location,object,33.27,3341,0
3,text,object,0.00,7503,0
4,target,int64,0.00,2,0


### Identificación de Variables

- `id` - Un identificador unico para cada tweet.
- `text` - El texto del tweet.
- `location` - La ubicación desde donde fue enviado el tweet.
- `keyword` - Una palabre clave del tweet.
- `target` - Etiqueta que denota si un tweet es sobre un desastre real o no